In [1]:
from pyuvdata import UVData, UVBeam
from pyuvdata import  utils as uvutils
import hera_pspec as hp
from hera_pspec.data import DATA_PATH
from hera_cal import redcal
from collections import OrderedDict as odict
from  hera_pspec import utils
from hera_pspec import uvpspec_utils as uvputils

In [2]:
import simpleqe as sqe
import matplotlib.gridspec as gridspec

In [3]:
from scipy.linalg import block_diag
from astropy.io import fits
from astropy import units, cosmology
from scipy.interpolate import interp2d, interp1d
from scipy import integrate
import sys, matplotlib, os
import numpy as np
import h5py
import time
import pickle
import copy
import itertools
import matplotlib.pyplot as plt
from matplotlib import colors, cm
cmap = cm.get_cmap('coolwarm')

In [4]:
from hera_pspec import UVPSpec, UVWindow, FTBeam

In [5]:
sys.path.insert(1, "/lustre/aoc/projects/hera/agorce/H1C_IDR3/")
import utils

In [10]:
from pspec_likelihood import *

In [11]:
from matplotlib import rc
rc('font',**{'family':'serif', 'serif':['times new roman'], 'size': 15})
rc('text', usetex=True)
rc('axes', linewidth = 1.5)

In [12]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [13]:
cosmo = cosmology.Planck18

In [14]:
path_to_uvp = '/lustre/aoc/projects/hera/agorce/tests_pspec_like/exact_wf/'

In [15]:
def theory_model(z, k, params):
    return params[0]*(1.+z)/k**params[1]

In [16]:
def bias(k, z, little_h=True):
    return np.ones(k.shape)

In [17]:
def bias_prior(params):
    return 1.

In [18]:
field = 'C'
spw = 0

# uvp_cosmo = UVPSpec()
# uvp_cosmo.read_hdf5(f'{validation_folder}Deltasq_Band_{spw+1}_Field_{field}.h5')

filename = f'{path_to_uvp}sph_xtk_idr3_field{field}_band{spw+1}.hdf5'
# filename = path_to_uvp+'u_xtk_idr3_exact_wf_field{}.hdf5'.format(field)

sph_pk = UVPSpec()
sph_pk.read_hdf5(filename)

u_xtk = UVPSpec()
u_xtk.read_hdf5(path_to_uvp+'u_xtk_idr3_field{}.hdf5'.format(field))

In [33]:
sph_pk.data_array[0].shape

(1, 40, 1)

In [19]:
sph_pk.convert_to_deltasq(inplace=True)

In [20]:
little_h = 'h^-3' in sph_pk.norm_units
print(little_h)

True


In [21]:
spw

0

In [22]:
band_index = spw
spw_index = sph_pk.spw_array[band_index]
spw_frequencies = sph_pk.get_spw_ranges()[spw_index][:2]
redshift = sph_pk.cosmo.f2z(np.mean(spw_frequencies))
print(redshift)

10.345518635132605


In [23]:
avg_nu = np.mean(sph_pk.freq_array[sph_pk.spw_to_freq_indices(spw)])
avg_z = sph_pk.cosmo.f2z(avg_nu)
print(avg_z)

10.349945290862271


In [24]:
"Spherically averaged with hera_pspec" in sph_pk.history

True

In [25]:
sph_pk.get_kperps(spw)

array([3.72335002e-12])

In [26]:
n_bands = len(u_xtk.get_all_keys())
print(n_bands, len(u_xtk.get_kperps(0)), len(np.unique(u_xtk.get_kperps(0))))
# print(u_xtk.get_kperps(0), len(u_xtk.get_red_blpairs()))

158 79 79


In [27]:
band_key = sph_pk.get_all_keys()[band_index]
ps = sph_pk.get_data(band_key).real.flatten()
ps2 = sph_pk.data_array[spw].real.flatten() 
print(ps.shape, ps2.shape, np.all(ps==ps2))

(40,) (40,) True


In [28]:
wf_3d = sph_pk.get_window_function(band_key)
window_function = block_diag(*wf_3d)
print(np.all(window_function==sph_pk.window_function_array[spw].squeeze()))

True


In [29]:
cosmology.units.littleh

Unit("littleh")

In [30]:
dmi = DataModelInterface(cosmology=cosmo,
                         redshift=redshift,
                         power_spectrum=sph_pk.data_array[spw].real.flatten() * units.mK**2,
                         window_function=sph_pk.window_function_array[spw].squeeze(),
                         covariance=sph_pk.cov_array_real[spw].squeeze()*units.mK**4,
                         kpar_bins_obs=sph_pk.get_kparas(spw) * cosmology.units.littleh / units.Mpc,
                         kpar_bins_theory=sph_pk.get_kparas(spw)* cosmology.units.littleh / units.Mpc,
                         kpar_widths_theory=None,
                         kperp_bins_obs=None,
                         kperp_bins_theory=None,
                         kperp_widths_theory=None,
                         window_integration_rule='midpoint',
                         theory_uses_little_h=little_h,
                         theory_uses_spherical_k=True,
                         theory_model=theory_model,
                         sys_model=None)

In [31]:
like = MarginalizedLinearPositiveSystematics(model=dmi)

/lustre/aoc/projects/hera/agorce/pspec_likelihood/src/pspec_likelihood/likelihood.py:618: UserWarning: Your covariance matrix is not diagonal. The MarginalizedLinearPositiveSystematics class requires diagonal covariance. Forcing it...
  warnings.warn(


In [32]:
like.loglike([10., 2.],
             [])

/lustre/aoc/projects/hera/agorce/pspec_likelihood/src/pspec_likelihood/likelihood.py:638: UserWarning: Warning: Ignoring data in positions (array([0, 1, 2]),) as the variance is zero
  warnings.warn(


UnitConversionError: Can only apply 'subtract' function to dimensionless quantities when other argument is not a quantity (unless the latter is all zero/infinity/nan)

In [27]:
np.all(dmi.power_spectrum == dmi4pspec.power_spectrum)

False

In [26]:
dmi4pspec = DataModelInterface.from_uvpspec(cosmology=cosmo,
                                            uvp=sph_pk, 
                                            band_index=spw,
                                            theory_uses_spherical_k=True,
                                            theory_model=theory_model,
                                            kpar_bins_theory=sph_pk.get_kparas(spw),
                                            window_integration_rule='midpoint',
                                            theory_uses_little_h=little_h,
                                            sys_model=None)

In [196]:
dmi.power_spectrum

<Quantity [      0.        ,       0.        ,       0.        ,
             80858.1557112 ,   81329.32832972,   90504.2068348 ,
            107614.96299835,   99195.38297638,  108445.1323706 ,
             99816.18130349,  103286.34348106,  151877.84512877,
            149367.80722872,  145560.31115762,  102137.46844667,
             89120.35018827,   77972.44196781,   86815.84509719,
             49130.84068066, -105740.81906454,  -20284.66735989,
             41902.49333536,   42071.74757227,   25750.82750968,
            297394.04877711,  256265.73981344,  -94613.20594183,
           -191752.59778745, -295577.12498071,  -78478.34603521,
            -79204.97326979,  671671.1024984 ,  728976.82580598,
            -67311.59368981,  -44299.26967549,  -21242.0572551 ,
           -315526.28902426,  221313.46481762,  186053.4940889 ,
            -39199.88159653] mK2>

In [197]:
dmi4pspec.power_spectrum

<Quantity [     0.        ,      0.        ,      0.        ,
            80858.1557112 ,  81329.32832972,  90504.2068348 ,
           107614.96299835,  99195.38297638, 108445.1323706 ,
            99816.18130349, 103286.34348106, 151877.84512877,
           149367.80722872, 145560.31115762, 102137.46844667,
            89120.35018827,  77972.44196781,  86815.84509719,
            49130.84068066,      0.        ,      0.        ,
            41902.49333536,  42071.74757227,  25750.82750968,
           297394.04877711, 256265.73981344,      0.        ,
                0.        ,      0.        ,      0.        ,
                0.        , 671671.1024984 , 728976.82580598,
                0.        ,      0.        ,      0.        ,
                0.        , 221313.46481762, 186053.4940889 ,
                0.        ] mK2>

In [191]:
for key in list(dmi.__dict__.keys()):
    if not np.all(dmi.__dict__[key]==dmi4pspec.__dict__[key]):
        print(key)

power_spectrum


KeyError: 'spherical_kbins_obs'

In [184]:
np.all(dmi.window_function == dmi4pspec.window_function)

True

In [185]:
dmi == dmi4pspec

False

In [36]:
a = np.diff(sph_pk.get_kparas(spw)).mean()*np.ones(sph_pk.get_kparas(spw).shape)
a

array([0.06185152, 0.06185152, 0.06185152, 0.06185152, 0.06185152,
       0.06185152, 0.06185152, 0.06185152, 0.06185152, 0.06185152,
       0.06185152, 0.06185152, 0.06185152, 0.06185152, 0.06185152,
       0.06185152, 0.06185152, 0.06185152, 0.06185152, 0.06185152,
       0.06185152, 0.06185152, 0.06185152, 0.06185152, 0.06185152,
       0.06185152, 0.06185152, 0.06185152, 0.06185152, 0.06185152,
       0.06185152, 0.06185152, 0.06185152, 0.06185152, 0.06185152,
       0.06185152, 0.06185152, 0.06185152, 0.06185152, 0.06185152])